In [4]:
# ==============================================================================
# @title 1. Impor Library
# ==============================================================================
# Jalankan sel ini untuk mengimpor library yang dibutuhkan.
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')
print("✅ Library berhasil diimpor.")

# ==============================================================================
# @title 2. Konfigurasi Path dan Variabel
# ==============================================================================
# Tentukan path untuk data mentah, data cuaca, dan folder output.
SOURCE_RAW_DIR = 'sumber_data_raw'
WEATHER_DATA_FILE = 'data_cuaca_merged.csv'
OUTPUT_DIR = 'sumber_data'

# Pastikan folder yang diperlukan ada
os.makedirs(SOURCE_RAW_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Folder sumber data mentah: '{SOURCE_RAW_DIR}'")
print(f"📁 File data cuaca: '{WEATHER_DATA_FILE}'")
print(f"📁 Folder output hasil: '{OUTPUT_DIR}'")

# ==============================================================================
# @title 3. Unggah Data Anda
# ==============================================================================
# PENTING: Sebelum melanjutkan, unggah data Anda ke Google Colab.
#
# 1. Di panel file sebelah kiri, temukan folder 'sumber_data_raw'.
#    - Klik kanan -> 'Upload' -> Unggah semua folder perangkat Anda (misal: witel, opmc).
# 2. Unggah file 'data_cuaca_merged.csv' ke direktori utama (di level yang sama
#    dengan folder 'sumber_data_raw').
#
# Setelah semua file dan folder terunggah, Anda bisa melanjutkan ke sel berikutnya.

print("✅ Sel ini siap.")
print(f"Pastikan Anda telah mengunggah data ke '{SOURCE_RAW_DIR}' dan file '{WEATHER_DATA_FILE}'.")

# ==============================================================================
# @title 4. Memuat dan Memproses Data Cuaca
# ==============================================================================
# Sel ini akan memuat file CSV data cuaca dan mempersiapkannya untuk digabung.

print(f"Memuat data cuaca dari '{WEATHER_DATA_FILE}'...")
try:
    df_weather = pd.read_csv(WEATHER_DATA_FILE)

    # Mengubah kolom waktu menjadi datetime dan menjadikannya index
    df_weather['time'] = pd.to_datetime(df_weather['time'])
    df_weather.set_index('time', inplace=True)

    # Mengganti nama kolom agar lebih mudah dibaca dan konsisten
    weather_column_mapping = {
        'relative_humidity_2m (%)': 'Relative Humidity',
        'dew_point_2m (°C)': 'Dew Point',
        'apparent_temperature (°C)': 'Apparent Temperature',
        'precipitation (mm)': 'Precipitation',
        'pressure_msl (hPa)': 'Pressure MSL',
        'surface_pressure (hPa)': 'Surface Pressure',
        'evapotranspiration (mm)': 'Evapotranspiration',
        'vapour_pressure_deficit (kPa)': 'Vapour Pressure Deficit',
        'wind_speed_10m (km/h)': 'Wind Speed',
        'wind_direction_10m (°)': 'Wind Direction',
        'wind_gusts_10m (km/h)': 'Wind Gusts',
        'soil_temperature_18cm (°C)': 'Soil Temperature',
        'sunshine_duration (s)': 'Sunshine Duration',
        'uv_index ()': 'UV Index',
        'direct_radiation (W/m²)': 'Direct Radiation',
        'temperature_2m (°C)': 'Temperature',
        'showers (mm)': 'Showers',
        'cloud_cover (%)': 'Cloud Cover',
        'weather_description': 'Weather Description'
    }
    df_weather.rename(columns=weather_column_mapping, inplace=True)

    print("✅ Data cuaca berhasil dimuat dan diproses.")
    print("Contoh data cuaca:")
    print(df_weather.head(3))

except FileNotFoundError:
    print(f"❌ ERROR: File '{WEATHER_DATA_FILE}' tidak ditemukan. Mohon unggah file tersebut.")
    df_weather = None
except Exception as e:
    print(f"❌ ERROR: Terjadi kesalahan saat memproses data cuaca: {e}")
    df_weather = None


# ==============================================================================
# @title 5. Proses Utama: Integrasi Data Perangkat dan Cuaca
# ==============================================================================
# Sel ini akan melakukan iterasi melalui semua file di 'sumber_data_raw',
# membersihkan duplikat, menghitung fitur baru, dan menggabungkannya.

if df_weather is not None:
    print("\nMemulai proses integrasi data...")
    for root, dirs, files in os.walk(SOURCE_RAW_DIR):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f"\n{'='*50}\nMemproses: {file_path}\n{'='*50}")

                try:
                    # 1. Memuat data perangkat
                    df_device = pd.read_csv(file_path)
                    print(f"   - Jumlah kolom sebelum pembersihan: {len(df_device.columns)}")

                    # 2. Logika Cerdas untuk Membersihkan Kolom Duplikat
                    cols_to_drop = []
                    cols_to_rename = {}
                    original_cols = df_device.columns.tolist()

                    for col_name in original_cols:
                        if col_name.endswith('.1'):
                            base_name = col_name[:-2]
                            if base_name in original_cols:
                                cols_to_drop.append(base_name)
                                cols_to_rename[col_name] = base_name
                    
                    if cols_to_drop:
                        df_device.drop(columns=cols_to_drop, inplace=True)
                        df_device.rename(columns=cols_to_rename, inplace=True)
                        print(f"   - Berhasil membersihkan dan me-rename {len(cols_to_drop)} kolom duplikat.")
                    else:
                        df_device = df_device.loc[:, ~df_device.columns.duplicated(keep='last')]

                    print(f"   - Jumlah kolom setelah pembersihan: {len(df_device.columns)}")
                    
                    df_device['id_time'] = pd.to_datetime(df_device['id_time'])

                    # 3. PEMBARUAN: Menghitung Fitur Elektrikal dengan Logika Baru
                    
                    # Kolom 'id_stand_energy_kirim' WAJIB ada
                    if 'id_stand_energy_kirim' in df_device.columns:
                        energy_col = df_device['id_stand_energy_kirim'].replace(0, np.nan)
                        df_device['Konsumsi Energi'] = energy_col.diff()
                        df_device.loc[df_device['Konsumsi Energi'] < 0, 'Konsumsi Energi'] = np.nan
                        print("   - Fitur 'Konsumsi Energi' berhasil dibuat.")
                    else:
                        print("   - ❌ KRITIS: Kolom 'id_stand_energy_kirim' tidak ditemukan. File ini akan dilewati.")
                        continue # Lanjut ke file berikutnya

                    # Fitur lain dibuat jika kolomnya ada
                    if all(col in df_device.columns for col in ['id_v1', 'id_v2', 'id_v3']):
                        df_device['Voltage'] = df_device[['id_v1', 'id_v2', 'id_v3']].mean(axis=1)
                        print("   - Fitur 'Voltage' berhasil dibuat.")
                    else:
                        print("   - ⚠️ PERINGATAN: Kolom voltage ('id_v1', 'id_v2', 'id_v3') tidak lengkap.")

                    if all(col in df_device.columns for col in ['id_i1', 'id_i2', 'id_i3']):
                        df_device['Current'] = df_device['id_i1'] + df_device['id_i2'] + df_device['id_i3']
                        print("   - Fitur 'Current' berhasil dibuat.")
                    else:
                        print("   - ⚠️ PERINGATAN: Kolom current ('id_i1', 'id_i2', 'id_i3') tidak lengkap.")

                    if 'id_power_factor' in df_device.columns:
                        df_device.rename(columns={'id_power_factor': 'Power Factor'}, inplace=True)
                        print("   - Fitur 'Power Factor' berhasil dibuat.")
                    else:
                        print("   - ⚠️ PERINGATAN: Kolom 'id_power_factor' tidak ditemukan.")


                    # 4. Menggabungkan dengan Data Cuaca
                    df_merged = pd.merge(
                        df_device, df_weather,
                        left_on='id_time', right_index=True, how='left'
                    )
                    print(f"   - Berhasil digabung dengan data cuaca. Total baris: {len(df_merged)}")

                    # 5. Menyimpan Hasil
                    relative_path = os.path.relpath(root, SOURCE_RAW_DIR)
                    output_folder = os.path.join(OUTPUT_DIR, relative_path)
                    os.makedirs(output_folder, exist_ok=True)
                    output_filename = f"Cleaned_{file}"
                    output_path = os.path.join(output_folder, output_filename)
                    df_merged.to_csv(output_path, index=False)
                    print(f"   -> Hasil disimpan ke: {output_path}")

                except Exception as e:
                    print(f"   - ❌ ERROR saat memproses file {file}: {e}")

    print("\n\n🏁 Proses Selesai. Semua file telah diproses dan disimpan di folder 'sumber_data'.")
else:
    print("\nProses tidak dapat dilanjutkan karena data cuaca gagal dimuat.")



✅ Library berhasil diimpor.
📁 Folder sumber data mentah: 'sumber_data_raw'
📁 File data cuaca: 'data_cuaca_merged.csv'
📁 Folder output hasil: 'sumber_data'
✅ Sel ini siap.
Pastikan Anda telah mengunggah data ke 'sumber_data_raw' dan file 'data_cuaca_merged.csv'.
Memuat data cuaca dari 'data_cuaca_merged.csv'...
✅ Data cuaca berhasil dimuat dan diproses.
Contoh data cuaca:
                     Relative Humidity  Dew Point  Apparent Temperature  \
time                                                                      
2024-01-01 00:00:00                 88       24.9                  33.5   
2024-01-01 01:00:00                 89       24.9                  33.2   
2024-01-01 02:00:00                 90       24.8                  33.0   

                     Precipitation  Pressure MSL  Surface Pressure  \
time                                                                 
2024-01-01 00:00:00            0.0        1009.5            1006.6   
2024-01-01 01:00:00            0.0      